# Lấy dữ liệu

In [1]:
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r feature_names

# Thay đổi dữ liệu để phù hợp

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression

# Reshape data
X_2d = X.reshape(-1, X.shape[-1])
y_reshaped = np.repeat(y, 20)

# Dùng hàm mutual_info_regression
# để đánh giá điểm cho các feature

In [3]:
# Compute information gain for each feature
info_gain = mutual_info_regression(X_2d, y_2d)

# Sắp xếp và lấy index của top k phần tử tốt nhất

In [25]:
# Sort the features based on information gain in descending order
sorted_features_indices = np.argsort(info_gain)[::-1]

# Select the top k features (adjust k as needed)
k = 48
selected_features_indices = sorted_features_indices[:k]

In [31]:
selected_X = X[:,:,selected_features_indices]
selected_X_test = X_test[:,:,selected_features_indices]

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(20, k, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 20, 48, 32)        832       
                                                                 
 conv2d_17 (Conv2D)          (None, 20, 48, 32)        25632     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 24, 32)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 10, 24, 32)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 10, 24, 64)        51264     
                                                                 
 conv2d_19 (Conv2D)          (None, 10, 24, 64)        102464    
                                                      

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(selected_X, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 30s 33ms/step - loss: 0.9293 - accuracy: 0.6392
Epoch 2/10
823/823 [==============================] - 28s 34ms/step - loss: 0.5356 - accuracy: 0.8084
Epoch 3/10
823/823 [==============================] - 29s 35ms/step - loss: 0.4101 - accuracy: 0.8597
Epoch 4/10
823/823 [==============================] - 33s 41ms/step - loss: 0.3618 - accuracy: 0.8750
Epoch 5/10
823/823 [==============================] - 29s 35ms/step - loss: 0.3453 - accuracy: 0.8794
Epoch 6/10
823/823 [==============================] - 29s 35ms/step - loss: 0.3097 - accuracy: 0.8890
Epoch 7/10
823/823 [==============================] - 29s 36ms/step - loss: 0.3005 - accuracy: 0.8928
Epoch 8/10
823/823 [==============================] - 28s 34ms/step - loss: 0.2859 - accuracy: 0.8976
Epoch 9/10
823/823 [==============================] - 31s 37ms/step - loss: 0.2608 - accuracy: 0.9043
Epoch 10/10
823/823 [==============================] - 29s 35ms/step - loss: 0.253

# Dữ liệu gốc

In [15]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 19s 45ms/step - loss: 0.1635 - accuracy: 0.9389
Độ chính xác trên tập kiểm tra: 0.9388576745986938
412/412 [==============================] - 18s 44ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1955    7   31   41]
 [   0   19 2625    1  163]
 [   0  321   13 1402    4]
 [   0   14  191    0 3537]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.85      0.96      0.90      2034
           2       0.93      0.93      0.93      2808
           3       0.98      0.81      0.88      1740
           4       0.94      0.95      0.94      3742

    accuracy                           0.94     13166
   macro avg       0.94      0.93      0.93     13166
weighted avg       0.94      0.94      0.94     13166



# k = 48

In [30]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 9s 20ms/step - loss: 0.2049 - accuracy: 0.9197
Độ chính xác trên tập kiểm tra: 0.9197174310684204
412/412 [==============================] - 8s 20ms/step
Ma trận nhầm lẫn:
[[2841    1    0    0    0]
 [   0 1931   10   67   26]
 [   0   36 2620    3  149]
 [   0  217   11 1501   11]
 [   0   52  472    2 3216]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.86      0.95      0.90      2034
           2       0.84      0.93      0.88      2808
           3       0.95      0.86      0.91      1740
           4       0.95      0.86      0.90      3742

    accuracy                           0.92     13166
   macro avg       0.92      0.92      0.92     13166
weighted avg       0.92      0.92      0.92     13166

